In [144]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [145]:
file_path = "/home/hwang/SIB/visualization/example_log.csv"

x_names = ['L%d%c' % (i, c) for i in range(21) for c in ['x', 'y', 'z']]
y_names = ['a', 's', 'd', 'f']

col_names = x_names + y_names

df = pd.read_csv(file_path,names=col_names)

In [146]:
import sys, os
sys.path.append(os.pardir)

from visualization.smoother import gamma_smoothing, average_smoothing

In [147]:
df = average_smoothing(df)

/home/hwang/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [148]:
scaler = MinMaxScaler()

for column in x_names:
	df_column = df[column]
	df_column = df_column.values.reshape(-1, 1)
	df_column = scaler.fit_transform(df_column)

	for row in range(len(df)):
		df[column].iloc[row] = df_column[row]


pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/hwang/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/hwang/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/hwang/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


In [149]:
x = df.iloc[:,:63]
y = df.iloc[:,63:]

In [150]:
x_train, x_test = x.iloc[:500,:], x.iloc[500:,:]
y_train, y_test = y.iloc[:500,:], y.iloc[500:,:]

In [151]:
x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

In [152]:
x_train_t = x_train.reshape(x_train.shape[0],63,1)
x_test_t = x_test.reshape(x_test.shape[0],63,1)

In [157]:
model = Sequential()

model.add(LSTM(20, input_shape=(63,1)))
model.add(Dense(64))
model.add(Dense(4))
model.compile(loss='mean_squared_error',optimizer='adam')

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                1344      
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 260       
Total params: 3,364
Trainable params: 3,364
Non-trainable params: 0
_________________________________________________________________


In [161]:
model.fit(x_train_t, y_train, epochs=50)

Epoch 1/50


ValueError: Creating variables on a non-first call to a function decorated with tf.function.